In [97]:
from collections import defaultdict

In [98]:
def readFASTA(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [99]:
def readFASTQ(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [100]:
def approx_match_editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance for approximate match is the minimum value in the last row of the matrix
    return min(D[-1])

In [101]:
genome = readFASTA('chr1.GRCh38.excerpt.fasta')

In [102]:
#Week 3 question 1
approx_match_editDistance('GCTGATCGATCGTACG', genome)

3

In [103]:
#Week 3 question 2
approx_match_editDistance('GATTTACCAGATTGAG', genome)

2

In [104]:
reads, _ = readFASTQ('ERR266411_1.for_asm.fastq')

In [105]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [106]:
def OverlapPairs(reads, min_length):
    KMReadDict = defaultdict(set)
    
    for r in reads:
        for i in range(len(r)-min_length+1):
            KMReadDict[r[i:i+min_length]].add(r)
    
    o_pair = defaultdict(set)
    for r in reads:
        for v in KMReadDict[r[-min_length:]]:
                if r!=v:
                    if overlap(r, v, min_length):
                        o_pair[r].add(v)
    o_edge = 0
    for read in o_pair:
        o_edge += len(o_pair[read])
        
    return o_edge, len(o_pair)

In [107]:
%%time
edge, num = OverlapPairs(reads, 30)

Wall time: 4.99 s


In [108]:
#Week 3 question 3
print(edge)

904746


In [110]:
#Week 3 question 4
print(num)

7161
